In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"yazhini4225","key":"1a6ba5ceade148cb5a7831e69292e05b"}'}

In [ ]:
!pip install kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c playground-series-s5e8

playground-series-s5e8.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip playground-series-s5e8.zip

Archive:  playground-series-s5e8.zip
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import pandas as pd
df=pd.read_csv('train.csv')
df.head()

,id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,0,42,technician,married,secondary,no,7,no,no,cellular,25,aug,117,3,-1,0,unknown,0
1,1,38,blue-collar,married,secondary,no,514,no,no,unknown,18,jun,185,1,-1,0,unknown,0
2,2,36,blue-collar,married,secondary,no,602,yes,no,unknown,14,may,111,2,-1,0,unknown,0
3,3,27,student,single,secondary,no,34,yes,no,unknown,28,may,10,2,-1,0,unknown,0
4,4,26,technician,married,secondary,no,889,yes,no,cellular,3,feb,902,1,-1,0,unknown,1


In [ ]:
len(df)

750000

In [ ]:
df.shape

(750000, 18)

In [ ]:
df.isnull().sum()

,0
id,0
age,0
job,0
marital,0
education,0
default,0
balance,0
housing,0
loan,0
contact,0


so it is a cleaned data

In [ ]:
print(df['job'].unique())

['technician' 'blue-collar' 'student' 'admin.' 'management' 'entrepreneur'
 'self-employed' 'unknown' 'services' 'retired' 'housemaid' 'unemployed']


In [ ]:
df['job']=df['job'].map({'admin.':0,'blue-collar':1,'entrepreneur':2,'housemaid':3,'management':4,'retired':5,'self-employed':6,'services':7,'student':8,'technician':9,'unemployed':10,'unknown':11})

In [ ]:
print(df['education'].unique())

['secondary' 'primary' 'tertiary' 'unknown']


In [ ]:
df['education']=df['education'].map({'primary':0,'secondary':1,'tertiary':2,'unknown':3})

In [ ]:
print(df['default'].unique())

['no' 'yes']


In [ ]:
df['default']=df['default'].map({'no':0,'yes':1,'unknown':2})

In [ ]:
print(df['housing'].unique())

['no' 'yes']


In [ ]:
df['housing']=df['housing'].map({'no':0,'yes':1,'unknown':2})

In [ ]:
df['loan']=df['loan'].map({'no':0,'yes':1,'unknown':2})

In [ ]:
df['marital'].unique()

array(['married', 'single', 'divorced'], dtype=object)

In [ ]:
df['marital']=df['marital'].map({'divorced':0,'married':1,'single':2,'unknown':3})

In [ ]:
df.head()

,id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,0,42,9,1,1,0,7,0,0,cellular,25,aug,117,3,-1,0,unknown,0
1,1,38,1,1,1,0,514,0,0,unknown,18,jun,185,1,-1,0,unknown,0
2,2,36,1,1,1,0,602,1,0,unknown,14,may,111,2,-1,0,unknown,0
3,3,27,8,2,1,0,34,1,0,unknown,28,may,10,2,-1,0,unknown,0
4,4,26,9,1,1,0,889,1,0,cellular,3,feb,902,1,-1,0,unknown,1


In [ ]:
df['poutcome'].unique()

array(['unknown', 'other', 'failure', 'success'], dtype=object)

In [ ]:
df['poutcome'].value_counts()

,count
poutcome,
unknown,672450
failure,45115
success,17691
other,14744


In [ ]:
df['poutcome']=df['poutcome'].map({'failure':0,'other':1,'success':2,'unknown':3})

In [ ]:
df.head()

,id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,0,42,9,1,1,0,7,0,0,cellular,25,aug,117,3,-1,0,3,0
1,1,38,1,1,1,0,514,0,0,unknown,18,jun,185,1,-1,0,3,0
2,2,36,1,1,1,0,602,1,0,unknown,14,may,111,2,-1,0,3,0
3,3,27,8,2,1,0,34,1,0,unknown,28,may,10,2,-1,0,3,0
4,4,26,9,1,1,0,889,1,0,cellular,3,feb,902,1,-1,0,3,1


In [ ]:
features=['age','job','marital','education','default','balance','housing','loan','pdays','previous','poutcome']
target='y'

In [ ]:
x=df[features]
y=df[target]

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_scaled=scaler.fit_transform(x_train)
x_test_scaled=scaler.transform(x_test)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
log_reg= LogisticRegression()
parameter={
    'C':[0.001,0.01,0.1,1,10,100,1000],
    'max_iter':[100,1000,2500,5000]
}
grid_search=GridSearchCV(log_reg,param_grid=parameter,cv=5,scoring='accuracy')
grid_search.fit(x_train_scaled,y_train)
best_parameers=grid_search.best_params_
print(best_parameers)


{'C': 0.1, 'max_iter': 100}


In [ ]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression(max_iter=100)
model.fit(x_train, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
print(model.score(x_test,y_test))

0.8774466666666667


In [ ]:
from sklearn.metrics import classification_report
y_pred=model.predict(x_test)
print(classification_report(y_test,y_pred))


              precision    recall  f1-score   support

           0       0.88      1.00      0.93    131795
           1       0.33      0.01      0.02     18205

    accuracy                           0.88    150000
   macro avg       0.60      0.50      0.48    150000
weighted avg       0.81      0.88      0.82    150000



In [ ]:
import numpy as np
new = [42, 1, 1, 0, 7, 25, 0, 117, 1, 0, 3]
new_array = np.array(new).reshape(1, -1)
prediction = model.predict(new_array)
print(prediction)


[0]


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [ ]:
import pandas as pd
test_df = pd.read_csv("test.csv")
test_df['job']=test_df['job'].map({'admin.':0,'blue-collar':1,'entrepreneur':2,'housemaid':3,'management':4,'retired':5,'self-employed':6,'services':7,'student':8,'technician':9,'unemployed':10,'unknown':11})
test_df['education']=test_df['education'].map({'primary':0,'secondary':1,'tertiary':2,'unknown':3})
test_df['default']=test_df['default'].map({'no':0,'yes':1,'unknown':2})
test_df['housing']=test_df['housing'].map({'no':0,'yes':1,'unknown':2})
test_df['loan']=test_df['loan'].map({'no':0,'yes':1,'unknown':2})
test_df['marital']=test_df['marital'].map({'divorced':0,'married':1,'single':2,'unknown':3})
test_df['poutcome']=test_df['poutcome'].map({'failure':0,'other':1,'success':2,'unknown':3})
if 'y' in test_df.columns:
    test_df = test_df.drop(columns=['y'])
ids = test_df['id']
X_test = test_df[features]
y_proba = model.predict_proba(X_test)[:, 1]
submission = pd.DataFrame({
    'id': ids,
    'y': y_proba
})
submission.to_csv("submission.csv", index=False)
print("Submission file saved as submission.csv")

Submission file saved as submission.csv


In [ ]:
from google.colab import files
files.download('submission.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>